# Day 22: Monkey Map

[https://adventofcode.com/2022/day/22](https://adventofcode.com/2022/day/22)

## Description

### Part One

The monkeys take you on a surprisingly easy trail through the jungle. They're even going in roughly the right direction according to your handheld device's Grove Positioning System.

As you walk, the monkeys explain that the grove is protected by a _force field_. To pass through the force field, you have to enter a password; doing so involves tracing a specific _path_ on a strangely-shaped board.

At least, you're pretty sure that's what you have to do; the elephants aren't exactly fluent in monkey.

The monkeys give you notes that they took when they last saw the password entered (your puzzle input).

For example:

            ...#
            .#..
            #...
            ....
    ...#.......#
    ........#...
    ..#....#....
    ..........#.
            ...#....
            .....#..
            .#......
            ......#.
    
    10R5L5R10L4R5L5
    

The first half of the monkeys' notes is a _map of the board_. It is comprised of a set of _open tiles_ (on which you can move, drawn `.`) and _solid walls_ (tiles which you cannot enter, drawn `#`).

The second half is a description of _the path you must follow_. It consists of alternating numbers and letters:

*   A _number_ indicates the _number of tiles to move_ in the direction you are facing. If you run into a wall, you stop moving forward and continue with the next instruction.
*   A _letter_ indicates whether to turn 90 degrees _clockwise_ (`R`) or _<span title="Or &quot;anticlockwise&quot;, if you're anti-counterclockwise.">counterclockwise</span>_ (`L`). Turning happens in-place; it does not change your current tile.

So, a path like `10R5` means "go forward 10 tiles, then turn clockwise 90 degrees, then go forward 5 tiles".

You begin the path in the leftmost open tile of the top row of tiles. Initially, you are facing _to the right_ (from the perspective of how the map is drawn).

If a movement instruction would take you off of the map, you _wrap around_ to the other side of the board. In other words, if your next tile is off of the board, you should instead look in the direction opposite of your current facing as far as you can until you find the opposite edge of the board, then reappear there.

For example, if you are at `A` and facing to the right, the tile in front of you is marked `B`; if you are at `C` and facing down, the tile in front of you is marked `D`:

            ...#
            .#..
            #...
            ....
    ...#.D.....#
    ........#...
    B.#....#...A
    .....C....#.
            ...#....
            .....#..
            .#......
            ......#.
    

It is possible for the next tile (after wrapping around) to be a _wall_; this still counts as there being a wall in front of you, and so movement stops before you actually wrap to the other side of the board.

By drawing the _last facing you had_ with an arrow on each tile you visit, the full path taken by the above example looks like this:

            >>v#    
            .#v.    
            #.v.    
            ..v.    
    ...#...v..v#    
    >>>v...>#.>>    
    ..#v...#....    
    ...>>>>v..#.    
            ...#....
            .....#..
            .#......
            ......#.
    

To finish providing the password to this strange input device, you need to determine numbers for your final _row_, _column_, and _facing_ as your final position appears from the perspective of the original map. Rows start from `1` at the top and count downward; columns start from `1` at the left and count rightward. (In the above example, row 1, column 1 refers to the empty space with no tile on it in the top-left corner.) Facing is `0` for right (`>`), `1` for down (`v`), `2` for left (`<`), and `3` for up (`^`). The _final password_ is the sum of 1000 times the row, 4 times the column, and the facing.

In the above example, the final row is `6`, the final column is `8`, and the final facing is `0`. So, the final password is 1000 \* 6 + 4 \* 8 + 0: _`6032`_.

Follow the path given in the monkeys' notes. _What is the final password?_

In [6]:
from pathlib import Path
from typing import Generator
import re

TEST = """        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5""".splitlines()

EXPECTED_1 = 6032
EXPECTED_2 = None

DATA = Path("input/data22.txt").read_text().splitlines()


def parse(data: list[str]) -> tuple[list[str], str]:
    rows = []
    for line in data[:-2]:
        if not line:
            continue
        rows.append(line)

    steps = [int(s) if s.isdigit() else s for s in re.split("(L|R)", data[-1])]
    return rows, steps


RIGHT, DOWN, LEFT, UP = 0, 1, 2, 3


def score_1(data: list[str]) -> int:
    grid, steps = parse(data)
    height = len(grid)
    width = max(len(r) for r in grid)
    left = [len(r) - len(r.lstrip()) for r in grid]
    right = [len(r) for r in grid]
    top = [
        min(y for y in range(height) if left[y] <= x < right[y]) for x in range(width)
    ]
    bottom = [
        max(y for y in range(height) if left[y] <= x < right[y]) + 1
        for x in range(width)
    ]

    def next(x, y, face):
        if face == RIGHT:
            x += 1
            if x >= right[y]:
                return left[y], y
        elif face == DOWN:
            y += 1
            if y >= bottom[x]:
                return x, top[x]
        elif face == LEFT:
            x -= 1
            if x < left[y]:
                return right[y] - 1, y
        else:
            y -= 1
            if y < top[x]:
                return x, bottom[x] - 1
        return x, y

    face = RIGHT
    x = left[0]
    y = 0
    # print(x,y,face)

    for step in steps:
        if step == "R":
            face = (face + 1) % 4
        elif step == "L":
            face = (face - 1) % 4
        else:
            for i in range(step):
                nx, ny = next(x, y, face)
                # print(x,y,face,nx,ny)
                if grid[ny][nx] == "#":
                    break
                x, y = nx, ny
        # print(x,y,face)

    return 1000 * (y + 1) + 4 * (x + 1) + face


test_score = score_1(TEST)
print("test", test_score)
assert test_score == EXPECTED_1
print("part 1", score_1(DATA))

test 6032
part 1 1428


<div class="alert alert-info">Easy start to the day, just a bit fiddly getting rid of all the off by one errors.</div>


### Part Two

As you reach the force field, you think you hear some Elves in the distance. Perhaps they've already arrived?

You approach the strange _input device_, but it isn't quite what the monkeys drew in their notes. Instead, you are met with a large _cube_; each of its six faces is a square of 50x50 tiles.

To be fair, the monkeys' map _does_ have six 50x50 regions on it. If you were to _carefully fold the map_, you should be able to shape it into a cube!

In the example above, the six (smaller, 4x4) faces of the cube are:

            1111
            1111
            1111
            1111
    222233334444
    222233334444
    222233334444
    222233334444
            55556666
            55556666
            55556666
            55556666
    

You still start in the same position and with the same facing as before, but the _wrapping_ rules are different. Now, if you would walk off the board, you instead _proceed around the cube_. From the perspective of the map, this can look a little strange. In the above example, if you are at A and move to the right, you would arrive at B facing down; if you are at C and move down, you would arrive at D facing up:

            ...#
            .#..
            #...
            ....
    ...#.......#
    ........#..A
    ..#....#....
    .D........#.
            ...#..B.
            .....#..
            .#......
            ..C...#.
    

Walls still block your path, even if they are on a different face of the cube. If you are at E facing up, your movement is blocked by the wall marked by the arrow:

            ...#
            .#..
         -->#...
            ....
    ...#..E....#
    ........#...
    ..#....#....
    ..........#.
            ...#....
            .....#..
            .#......
            ......#.
    

Using the same method of drawing the _last facing you had_ with an arrow on each tile you visit, the full path taken by the above example now looks like this:

            >>v#    
            .#v.    
            #.v.    
            ..v.    
    ...#..^...v#    
    .>>>>>^.#.>>    
    .^#....#....    
    .^........#.    
            ...#..v.
            .....#v.
            .#v<<<<.
            ..v...#.
    

The final password is still calculated from your final position and facing from the perspective of the map. In this example, the final row is `5`, the final column is `7`, and the final facing is `3`, so the final password is 1000 \* 5 + 4 \* 7 + 3 = _`5031`_.

Fold the map into a cube, _then_ follow the path given in the monkeys' notes. _What is the final password?_


In [7]:

cube = """
        1111
        1111
        1111
        1111
222233334444
222233334444
222233334444
222233334444
        55556666
        55556666
        55556666
        55556666"""

cube2 = """
    22221111
    22221111
    22221111
    22221111
    3333  
    3333
    3333
    3333
44445555
44445555
44445555
44445555
6666
6666
6666
6666
"""

<div class="alert alert-info">I drew out nets of the test cube and the actual cube. This didn't help a lot so I also sketched the actual cube on paper and cut it and folded it and marked the corresponding edges. Then I created some mappings for current direction and target location where it has to wrap and the corresponding point and direction.</div>

In [8]:

def cube_map2(face: int) -> dict[int, dict[tuple[int, int], tuple[int, int, int]]]:
    mapping = {LEFT: {}, RIGHT: {}, UP: {}, DOWN: {}}
    # bottom of 1 -> right of 3 moving left
    # right of 3 -> bottom of 1 moving up
    for x in range(face):
        mapping[DOWN][2 * face + x, face] = (2 * face - 1, face + x, LEFT)
        mapping[RIGHT][2 * face, face + x] = (2 * face + x, face - 1, UP)

    # right of 1 -> right of 5 moving left REVERSED
    # right of 5 -> right of 1 moving left REVERSED
    for y in range(face):
        mapping[RIGHT][3 * face, y] = (2 * face - 1, 3 * face - y - 1, LEFT)
        mapping[RIGHT][2 * face, 3 * face - y - 1] = (3 * face - 1, y, LEFT)

    # bottom of 5 -> right of 6 moving left
    # right of 6 -> bottom of 5 moving up
    for x in range(face):
        mapping[DOWN][face + x, 3 * face] = (face - 1, 3 * face + x, LEFT)
        mapping[RIGHT][face, 3 * face + x] = (face + x, 3 * face - 1, UP)

    # top of 1 moving up -> bottom of 6 moving up
    # bottom of 6 -> top of 1 moving down
    for x in range(face):
        mapping[UP][2 * face + x, -1] = (x, 4 * face - 1, UP)
        mapping[DOWN][x, 4 * face] = (2 * face + x, 0, DOWN)

    # top of 2 -> left of 6 moving right
    # left of 6 -> top of 2 moving down
    for x in range(face):
        mapping[UP][face + x, -1] = (0, 3 * face + x, RIGHT)
        mapping[LEFT][-1, 3 * face + x] = (face + x, 0, DOWN)

    # left of 2 -> left of 4 moving right REVERSED
    # left of 4 -> left of 2 moving right REVERSED
    for y in range(face):
        mapping[LEFT][face - 1, y] = (0, 3 * face - y - 1, RIGHT)
        mapping[LEFT][-1, 3 * face - y - 1] = (face, y, RIGHT)

    # left of 3 -> top of 4 moving down
    # top of 4 -> left of 3 moving right
    for y in range(face):
        mapping[LEFT][face - 1, face + y] = (y, 2 * face, DOWN)
        mapping[UP][y, 2 * face - 1] = (face, face + y, RIGHT)

    return mapping


def cube_map(face: int) -> dict[int, dict[tuple[int, int], tuple[int, int, int]]]:
    mapping = {LEFT: {}, RIGHT: {}, UP: {}, DOWN: {}}
    # right of 1 maps to right of 6 (moving left)
    # right of 6 maps to right of 1 moving left
    for y in range(face):
        mapping[RIGHT][3 * face, y] = (4 * face - 1, 3 * face - y - 1, LEFT)
        mapping[RIGHT][4 * face, 3 * face - y - 1] = (3 * face - 1, y, LEFT)

    # top of 1 maps to top of 2 (moving down)
    # top of 2 maps to top of 1 (moving down)
    for x in range(face):
        mapping[UP][x + 2 * face, -1] = (face - x - 1, face, DOWN)
        mapping[UP][face - x - 1, face - 1] = (x + 2 * face, 0, DOWN)

    # left of 1 maps to top of 3 (moving down)
    # top of 3 maps to left of 1 moving right
    for y in range(face):
        mapping[LEFT][2 * face - 1, y] = (face + y, face, DOWN)
        mapping[UP][face + y, face - 1] = (2 * face, y, RIGHT)

    # left of 2 maps to bottom of 6 (moving up)
    # bottom of 6 maps to left of 2 moving right
    for y in range(face):
        mapping[LEFT][-1, face + y] = (4 * face - y - 1, 3 * face - 1, UP)
        mapping[DOWN][4 * face - y - 1, 3 * face] = (0, face + y, RIGHT)

    # bottom of 2 maps to bottom of 5 (moving up)
    # bottom of 5 maps to bottom of 2 moving up
    for x in range(face):
        mapping[DOWN][x, 2 * face] = (3 * face - x - 1, 3 * face - 1, UP)
        mapping[DOWN][3 * face - x - 1, 3 * face] = (x, 2 * face - 1, UP)

    # bottom of 3 maps to left of 5 (moving right)
    # left of 5 maps to bottom of 3 moving up
    for x in range(face):
        mapping[DOWN][face + x, 2 * face] = (2 * face, 3 * face - x - 1, RIGHT)
        mapping[LEFT][2 * face - 1, 3 * face - x - 1] = (face + x, 2 * face - 1, UP)

    # top of 6 maps to right of 4 moving left
    # right of 4 maps to top of 6 moving down
    for x in range(face):
        mapping[UP][3 * face + x, 2 * face - 1] = (3 * face - 1, 2 * face - x - 1, LEFT)
        mapping[RIGHT][3 * face, 2 * face - x - 1] = (3 * face + x, 2 * face, DOWN)
    return mapping


<div class="alert alert-info">Finally the actual code:</div>

In [9]:
MARKERS = {UP: "^", RIGHT: ">", LEFT: "<", DOWN: "v"}


def score_2(data: list[str]) -> int:
    grid, steps = parse(data)
    height = len(grid)
    width = max(len(r) for r in grid)
    left = [len(r) - len(r.lstrip()) for r in grid]
    right = [len(r) for r in grid]
    top = [
        min(y for y in range(height) if left[y] <= x < right[y]) for x in range(width)
    ]
    bottom = [
        max(y for y in range(height) if left[y] <= x < right[y]) + 1
        for x in range(width)
    ]

    path = [list(r) for r in grid]

    if width > height:
        face_size = width // 4
        mapping = cube_map(face_size)
        x = 2 * face_size
    else:
        face_size = height // 4
        mapping = cube_map2(face_size)
        x = face_size

    # Validate mapping
    for _x in range(width):
        assert (_x, top[_x] - 1) in mapping[UP], f"up {_x}, {top[_x]-1}"
        assert (_x, bottom[_x]) in mapping[DOWN], f"down {_x}, {bottom[_x]}"
    for _y in range(height):
        assert (left[_y] - 1, _y) in mapping[LEFT], f"left {left[_y]-1}, {_y}"
        assert (right[_y], _y) in mapping[RIGHT], f"right {right[_y]} {_y}"

    f = face_size

    def next(x: int, y: int, face: int) -> tuple[int, int, int]:
        if face == RIGHT:
            return mapping[face].get((x + 1, y), (x + 1, y, face))
        elif face == DOWN:
            return mapping[face].get((x, y + 1), (x, y + 1, face))
        elif face == LEFT:
            return mapping[face].get((x - 1, y), (x - 1, y, face))

        return mapping[face].get((x, y - 1), (x, y - 1, face))

    face = RIGHT
    y = 0
    path = [list(r) for r in grid]
    path[y][x] = MARKERS[face]

    for step in steps:
        if step == "R":
            face = (face + 1) % 4
            path[y][x] = MARKERS[face]
        elif step == "L":
            face = (face - 1) % 4
            path[y][x] = MARKERS[face]
        else:
            for i in range(step):
                nx, ny, nface = next(x, y, face)

                if grid[ny][nx] == "#":
                    break

                x, y, face = nx, ny, nface
                path[y][x] = MARKERS[face]

    if face_size == 4:
        for r in path:
            print("".join(r))

    return 1000 * (y + 1) + 4 * (x + 1) + face


EXPECTED_2 = 5031

if EXPECTED_2 is not None:
    test_score = score_2(TEST)
    assert test_score == EXPECTED_2
    print("test", test_score)
    print("part 2", score_2(DATA))


        >>v#
        .#v.
        #.v.
        ..v.
...#..^...v#
.>>>>>^.#.>>
.^#....#....
.^........#.
        ...#..v.
        .....#v.
        .#v<<<<.
        ..v...#.
test 5031
part 2 142380


<div class="alert alert-info">It took me a while to get this working. The bug was in the `next()` function: when the location wasn't in the wrap-around map I was forgetting to increment the location so the cursor just sat on a single spot facing different directions. It wasn't until I got the test map in there and was able to print out the path I could see what I'd done. After fixing that the printed path matches the example and the output is the correct answer.</div>